In [ ]:
!pip install nltk
!pip install spacy
!pip install markovify
!pip install -m spacy download en

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 10.7 MB/s eta 0:00:00
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18605 sha256=ec94049727370e4796e742b42f0fb446384cabdb82a65dc0589d11e7a294e65a
  Stored in directory: /root/.cache/pip/wheels/ca/8c/c5/41413e24c484f883a100c63ca7b3b0362b7c6f6eb6d7c9cc7f
Successfully built markovify

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -m


In [ ]:
import spacy
import re
import markovify
import nltk
from nltk.corpus import gutenberg
import warnings
warnings.filterwarnings('ignore')
nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 94.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#inspect Gutenberg corpus
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [ ]:

macbeth = gutenberg.raw('shakespeare-macbeth.txt')
caesar = gutenberg.raw('shakespeare-caesar.txt')

print('\nRaw:\n', macbeth[:100])
print('\nRaw:\n', caesar[:100])


Raw:
 [The Tragedie of Macbeth by William Shakespeare 1603]


Actus Primus. Scoena Prima.

Thunder and Lig

Raw:
 [The Tragedie of Julius Caesar by William Shakespeare 1599]


Actus Primus. Scoena Prima.

Enter Fla


In [ ]:
#utility function for text cleaning
def text_cleaner(text):
  text = re.sub(r'--', ' ', text)
  text = re.sub('[\[].*?[\]]', '', text)
  text = re.sub(r'(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b','', text)
  text = ' '.join(text.split())
  return text

In [ ]:
#remove chapter indicator
macbeth = re.sub(r'Chapter \d+', '', macbeth)
caesar = re.sub(r'Chapter \d+', '', caesar)
#apply cleaning function to corpus
caesar = text_cleaner(caesar)
macbeth = text_cleaner(macbeth)

In [ ]:
#parse cleaned novels
import spacy
nlp = spacy.load("en_core_web_sm")
macbeth_doc = nlp(macbeth)
caesar_doc = nlp(caesar)

In [ ]:
macbeth_sents = ' '.join([sent.text for sent in macbeth_doc.sents if len(sent.text) > 1])
caesar_sents = ' '.join([sent.text for sent in caesar_doc.sents if len(sent.text) > 1])
shakespeare_sents =  macbeth_sents + caesar_sents
#inspect our text
print(shakespeare_sents)

Actus Primus. Scoena Prima. Thunder and Lightning. Enter three Witches.. When shall we three meet againe? In Thunder, Lightning, or in Raine?. When the Hurley-burley's done, When the Battaile's lost, and wonne. That will be ere the set of Sunne. Where the place?. Vpon the Heath. There to meet with Macbeth. I come, Gray-Malkin All. Padock calls anon: faire is foule, and foule is faire, Houer through the fogge and filthie ayre. Exeunt. Scena Secunda. Alarum within. Enter King Malcome, Donalbaine, Lenox, with attendants, meeting a bleeding Captaine. King. What bloody man is that? he can report, As seemeth by his plight, of the Reuolt The newest state Mal. This is the Serieant, Who like a good and hardie Souldier fought 'Gainst my Captiuitie: Haile braue friend; Say to the King, the knowledge of the Broyle, As thou didst leaue it Cap. Doubtfull it stood, As two spent Swimmers, that doe cling together, And choake their Art: The mercilesse Macdonwald (Worthie to be a Rebell, for to that The 

In [ ]:
generator_1 = markovify.Text(shakespeare_sents, state_size=3)

In [ ]:
for i in range(3):
  print(generator_1.make_short_sentence(max_chars=100))

Will you go see the order of your going, But go at once Len.
Speake if you can: what are you?. All haile Macbeth, that shalt be King hereafter Banq.
Beare with me, My heart is thirsty for that Noble pledge.


In [ ]:
class POSifiedText(markovify.Text):
   def word_split(self, sentence):
      return ['::'.join((word.orth_, word.pos_)) for word in nlp(sentence)]
   def word_join(self, words):
      sentence = ' '.join(word.split('::')[0] for word in words)
      return sentence
generator_2 = POSifiedText(shakespeare_sents, state_size=3)

In [ ]:
for i in range(3):
  print(generator_2.make_short_sentence(max_chars=100))

Comes the King forth I pray you do Brut .
Haue I in Conquest stretcht mine Arme so farre , that should be in that Caesar ?
Did I not meet thy Friends , and did take it from him Titin .
